# [Module 6.2] Kubeflow에서 요청하여 만든 SageMaker 모델 기반으로 추론 

이 노트북은 Kubeflow Pipeline에서 SageMaker로 Training job을 요청을 했고, <br>
생성한 모델 아티펙트를 가져와서 추론을 합니다.


**아래 일부 코드는 하드코드가 되어 있습니다. 
(예: training job, inference image 등) 
실제 환경 구성후에 실행시 변경 해야 합니다.**

---
이 노트북은 약 10분 정도 시간이 소요 됩니다.

In [1]:
!pip install -q tensorflow==2.1.0

In [2]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## SageMaker Train Model Invoked by KubeFlow

엔드포인트를 만들기 위해 아래 두가지를 준비 합니다.
- SageMaker Training Job을 가져와서 Model Artifact를 사용 합니다.
- "4.2.1.Make-Custom-Inference-Image-ECR.ipynb" 에서 생성하여 ECR에 등록한 추론 이미지를 가져옵니다.

In [4]:
model_data = 's3://sagemaker-ap-northeast-2-790639055451/bert-kf-output/TrainingJob-20210408140238-J5UG/output/model.tar.gz'
# model_data = '<>'
print('sagemaker train job from KubeFlow: \n', model_data)

sagemaker train job from KubeFlow: 
 s3://sagemaker-ap-northeast-2-790639055451/bert-kf-output/TrainingJob-20210408140238-J5UG/output/model.tar.gz


In [5]:
inference_image = '790639055451.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0-cpu'
# inference_image = <>
print("inference_image: ", inference_image)

inference_image:  790639055451.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0-cpu


## 엔드포인트 생성

In [ ]:
%%time

from sagemaker.tensorflow.serving import Model

import time
endpont_name    = f'kubeflow-custom-inference-image-endpoint-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'



model = Model(model_data= model_data,
              role=role,
              entry_point='inference.py',
              image_uri = inference_image
             ) 

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(
                             endpoint_name= endpont_name,
                             initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-

## Predictor Creation on the Endpoint

In [ ]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

In [ ]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [ ]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

In [ ]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

In [ ]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 5 이모티콘 추천

In [ ]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    tweets = [tweet]    
    predicted_classes = predictor.predict(tweets)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('-------------------------------------------')        
    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        ))    